In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import time

deflating = False

def print_volume_handler(unused_addr, args, volume):
    global deflating, deflateStartTime
    
    print("[{0}] ~ {1}".format(args, volume))
    
    if (volume > 1090 or deflating):
        client.send_message("/actuator/1/inflate", -80.0)   # Send float message between -100 and 100
        
        if deflating == False:
            deflating = True
            deflateStartTime = time.time()
        
        elif time.time() - deflateStartTime > 5:
            deflating = False
            
    elif deflating == False:
        client.send_message("/actuator/1/inflate", 80.0)   # Send float message between -100 and 100


deflateStartTime = time.time()
    
ip = "192.168.0.106"
port = 32000

client = SimpleUDPClient(ip, port)  # Create client
    
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/sensor/pressure", print_volume_handler, "Pressure")

server = osc_server.ThreadingOSCUDPServer(("192.168.0.106", 31000), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [2]:
from pythonosc.udp_client import SimpleUDPClient

ip = "192.168.0.106"
port = 32000

client = SimpleUDPClient(ip, port)  # Create client

client.send_message("/actuator/1/inflate", -0.0)


In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient


def print_volume_handler(unused_addr, args, volume):
    print("[{0}] ~ {1}".format(args, volume))
    
    client.send_message("/actuator/1/inflate", -100.0)   # Send float message between -100 and 100


    
ip = "192.168.0.106"
port = 32000

client = SimpleUDPClient(ip, port)  # Create client
    
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/sensor/pressure", print_volume_handler, "Pressure")

server = osc_server.ThreadingOSCUDPServer(("192.168.0.106", 31000), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [4]:
import time
time.time()

1580817874.956471

# Respiration Real-Time Analysis

In [1]:
import biosppy.signals.resp as resp
import numpy as np
import csv as csv
import json    
import matplotlib.pyplot as plt

def calc_resp_intervals(data, last_breath = False):
    processed_data = resp.resp(signal=data, sampling_rate=200, show=False)
    filtered_signal = processed_data[1]
    inst_resp_rate = processed_data[4]
    
    signal_diff = np.diff(filtered_signal)
    signal_signum = signal_diff > 0
    
    resp_changes = np.append(np.where(signal_signum[:-1] != signal_signum[1:])[0], [len(signal_signum) - 1])
    
    if not last_breath:
        
        resp_intervals = np.append([0], resp_changes)
        interval_lengths = np.diff(resp_intervals)
        interval_breathe_in = [signal_signum[i] for i in resp_changes]
        return interval_lengths, interval_breathe_in
    
    else:
        if len(resp_changes) > 1:
            last_interval = resp_changes[-1] - resp_changes[-2]
        else:
            last_interval = resp_changes[-1] 
                   
        return last_interval, signal_signum[resp_changes[-1]]

In [2]:
import time
from pythonosc.udp_client import SimpleUDPClient

resp_data = []
last_update = time.time()

update_freq = 0.5

riot_ip = '192.168.1.2'
riot_port = 8888
actuator_port = 12000
actuator_ip = '192.168.0.103'


client = SimpleUDPClient(actuator_ip, actuator_port) 

def process_riot_data(unused_addr, *values):
    global resp_data, last_update, client
        
    new_data = values[12]
    resp_data.append(new_data)
        
    if len(resp_data) > 200*10 and time.time() - last_update > update_freq:
        last_int, breathe_in = calc_resp_intervals(resp_data, last_breath = True)
                
        if breathe_in:
            print("Breathing in")
            client.send_message("/actuator/inflate", 100.0)
        else:
            print("Breathing out")
            client.send_message("/actuator/inflate", -100.0)
        
        last_update = time.time()
    
    # only save the last 5 min of data
    if len(resp_data) > 200 * 60 * 5:
        resp_data = resp_data[-200*60*5:]
    

In [3]:
from pythonosc import dispatcher
from pythonosc import osc_server

riot_dispatcher = dispatcher.Dispatcher()
riot_dispatcher.map("/*/raw", process_riot_data)

server = osc_server.ThreadingOSCUDPServer((riot_ip, riot_port), riot_dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

Serving on ('192.168.1.2', 8888)
Breathing out
Breathing out
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing outBreathing out

Breathing out
Breathing out
Breathing out
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing out
Breathing out
Breathing outBreathing out

Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing out
Breathing in
Breathing in
Breathing inBreathing in

Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in
Breathing in


KeyboardInterrupt: 

In [6]:
from pythonosc.udp_client import SimpleUDPClient

riot_ip = '192.168.0.103'
riot_port = 8888
actuator_port = 12000

client = SimpleUDPClient(riot_ip, actuator_port) 

client.send_message("/actuator/inflate", -0.0)